In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
CNV_df = pd.read_csv("/Users/oo/Desktop/final omics data/CNV.csv")
CNV_df = CNV_df.iloc[:, 1:]

In [ ]:
mRNA_df = pd.read_csv("/Users/oo/Desktop/final omics data/mRNA.csv")
mRNA_df = mRNA_df.iloc[:, 1:]
mRNA_df

In [ ]:
DNAm_df = pd.read_csv("/Users/oo/Desktop/final omics data/DNAm.csv")
DNAm_df = DNAm_df.iloc[:, 1:]

In [ ]:
miRNA_df = pd.read_csv("/Users/oo/Desktop/final omics data/miRNA.csv")
miRNA_df = miRNA_df.iloc[:, 1:]

In [ ]:
mRNA_df.iloc[:, 4:].value_counts()

In [ ]:
CNV_df[CNV_df['duration'] != 0].iloc[:, 4:]
mRNA_df[mRNA_df['duration'] != 0].iloc[:, 4:]
miRNA_df[miRNA_df['duration'] != 0].iloc[:, 4:]
DNAm_df[DNAm_df['duration'] != 0].iloc[:, 4:]

In [ ]:
from sklearn.preprocessing import normalize
import warnings
import numpy as np
warnings.filterwarnings('ignore')
mRNA_temp_df = mRNA_df.iloc[:, 4:]
miRNA_temp_df = miRNA_df.iloc[:, 4:]
DNAm_temp_df = DNAm_df.iloc[:, 4:]
CNV_temp_df = CNV_df.iloc[:, 4:]

mRNA_norm = normalize(mRNA_temp_df, axis=0)
miRNA_norm = normalize(miRNA_temp_df, axis=0)
DNAm_norm = normalize(DNAm_temp_df, axis=0)
CNV_norm = normalize(CNV_temp_df, axis=0)
omics_data = pd.DataFrame(np.hstack([mRNA_norm, miRNA_norm, DNAm_norm, CNV_norm]))
clinical_data = mRNA_df.iloc[:, :4]
omics_data

# Modeling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Input, Dropout, BatchNormalization, LeakyReLU
from keras.regularizers import l1_l2
from keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

In [ ]:
input_dim = omics_data.shape[1]
input_layer = Input(shape=(input_dim,))
encoded = Dense(1000, activation='relu', activity_regularizer=l1_l2(l1=0.001, l2=0.0001))(input_layer)
# encoded = LeakyReLU(alpha=0.01)(encoded)
encoded = Dropout(0.5)(encoded)
# encoded = BatchNormalization()(encoded)
encoder_output = Dense(500, activation='relu', activity_regularizer=l1_l2(l1=0.001, l2=0.0001), )(encoded)
# encoder_output = LeakyReLU(alpha=0.01)(encoder_output)
# encoder_output = Dropout(0.5)(encoder_output)
# encoder_output = BatchNormalization()(encoder_output)
decoded = Dense(1000, activation='relu',activity_regularizer=l1_l2(l1=0.001, l2=0.0001))(encoder_output)
# decoded = LeakyReLU(alpha=0.01)(decoded)
decoded = Dropout(0.5)(decoded)
# decoded = BatchNormalization()(decoded)

decoder_output = Dense(input_dim, activation='tanh')(decoded)
autoencoder = Model(input_layer, decoder_output)
encoder = Model(input_layer, encoder_output)

autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse'])
history = autoencoder.fit(omics_data, omics_data,
                epochs=3,
                batch_size=16,
                validation_split=0.3
           )
bottleneck_feat = encoder.predict(omics_data)
print(bottleneck_feat.shape)
print('0 ratio:', (bottleneck_feat == 0).sum() / (407*500))
encoder_df = pd.DataFrame(bottleneck_feat)
encoder_df

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans = kmeans.fit(encoder_df)
label = kmeans.predict(encoder_df)
encoder_df['label'] = label
final_data = pd.concat([clinical_data, encoder_df], axis=1)
final_feats = final_data.iloc[:, 2:]

In [ ]:
significant_features = []
from lifelines import CoxPHFitter
cph = CoxPHFitter()

coxPH_data = pd.concat([clinical_data[['vital_status', 'duration']], encoder_df], axis=1)
coxPH_data['vital_status'] = coxPH_data['vital_status'].astype('int')
np.random.seed(42)
numbers = np.arange(2, 502)
np.random.shuffle(numbers)
for i in numbers:
    
    cph.fit(coxPH_data.iloc[:, [0, 1, i]], duration_col='duration', event_col='vital_status',)
    summary = cph.summary
    # print(summary.index)
    if summary.loc[i-2, 'p'] < 0.05:
        significant_features.append((i-2, summary.loc[i-2, 'p']))
        
significant_features_df = pd.DataFrame(significant_features, columns=['Feature', 'P_Value'])
# significant_features_df.sort_values('Feature')
# coxPH_data
significant_features_df.shape
# cph.fit(coxPH_data.iloc[:, [0, 1, 20]], duration_col='duration', event_col='vital_status')
# coxPH_data.iloc[:, [0, 1, 23]]

In [ ]:
selected_feat = pd.concat([clinical_data[['case_submitter_id', 'vital_status', 'duration']], encoder_df[significant_features_df['Feature']]], axis=1)
selected_feat = selected_feat[selected_feat['duration'] != 0].reset_index(drop=True)
selected_feat.to_csv("/Users/oo/Desktop/final omics data/selected_feat.csv")


In [ ]:
selected_feat = pd.read_csv("/Users/oo/Desktop/final omics data/selected_feat.csv", index_col=0)
selected_feat.iloc[:, 3:]

In [ ]:
final_feats = pd.read_csv("/Users/oo/Desktop/final omics data/final_feats.csv", index_col=0)
final_feats


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import pandas as pd
final_feats = pd.read_csv("/Users/oo/Desktop/final omics data/final_feats.csv")
final_feats = final_feats.iloc[:, 1:]

selected_feat = pd.read_csv("/Users/oo/Desktop/final omics data/selected_feat.csv")
selected_feat = selected_feat.rename(columns = {'Unnamed: 0' : 'label'})
kmeans = KMeans(n_clusters=2, random_state=42)
kmeans = kmeans.fit(final_feats)
label = kmeans.predict(final_feats)
selected_feat['label'] = label
# selected_feat = selected_feat.drop(columns=['case_submitter_id'])
selected_feat.reset_index(drop=True, inplace=True)
final_feats = pd.concat([selected_feat[['label', 'vital_status', 'duration']], final_feats], axis=1)
final_feats['label'].value_counts()


In [ ]:
final_feats = pd.read_csv("/Users/oo/Desktop/final omics data/final_feats.csv")
final_feats = final_feats.iloc[:, 1:]
final_feats

In [ ]:
silhouette_scores = []
for n_clusters in range(2, 10):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(final_feats)
    score = silhouette_score(final_feats, kmeans.labels_)
    silhouette_scores.append(score)

plt.plot(range(2, 10), silhouette_scores)
plt.title('Silhouette Scores')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.show()

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.utils import concordance_index
from lifelines.statistics import logrank_test
from lifelines import CoxPHFitter

kmf1 = KaplanMeierFitter()
kmf2 = KaplanMeierFitter()
cph1 = CoxPHFitter()
cph2 = CoxPHFitter()



g1 = final_feats[final_feats['label'] == 0]
g2 = final_feats[final_feats['label'] == 1]



duration_1 = g1['duration']
duration_2 = g2['duration']

event_1 = g1['vital_status']
event_2 = g2['vital_status']

cph1.fit(g1.drop(columns='label'), event_col='vital_status', duration_col='duration')
cph2.fit(g2.drop(columns='label'), event_col='vital_status', duration_col='duration')


kmf1.fit(duration_1, event_1)
kmf2.fit(duration_2, event_2)


results = logrank_test(duration_1, duration_2, event_observed_A=event_1, event_observed_B=event_2)
p_value = results.p_value

c_index_1 = concordance_index(duration_1, -cph1.predict_partial_hazard(g1), event_1)
c_index_2 = concordance_index(duration_2, -cph2.predict_partial_hazard(g2), event_2)
c_index = (c_index_1 + c_index_2) / 2
print(f'log-rank P-value: {p_value}')
print(f'C-index: {c_index}\nC-index1: {c_index_1}\nC-index2: {c_index_2}')


In [ ]:
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt
kmf = KaplanMeierFitter()
# plt.figure(figsize=(12, 8))
for i in range(2):
    cluster_data = final_feats[final_feats['label'] == i]
    
    kmf.fit(cluster_data['duration'], event_observed=cluster_data['vital_status'], label=f'G{i+1}(n={cluster_data.shape[0]})')
    kmf.plot_survival_function()
    

# plt.figtext(0.6, 0.7, f"log-rank P-value: {p_value:.2e}\nC-index: {c_index:.2}", fontsize=10)
plt.title('Kaplan-Meier Curves by Cluster')
plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.show()

In [ ]:
final_feats.to_csv('/Users/oo/Desktop/final omics data/omics_data.csv')

In [ ]:
final_feats = pd.read_csv("/Users/oo/Desktop/final omics data/omics_data.csv")
final_feats

In [ ]:
final_CNV_df = CNV_df[CNV_df['duration'] != 0].drop(columns='case_id').reset_index(drop=True)
final_mRNA_df = mRNA_df[mRNA_df['duration'] != 0].drop(columns='case_id').reset_index(drop=True)
final_miRNA_df = miRNA_df[miRNA_df['duration'] != 0].drop(columns='case_id').reset_index(drop=True)
final_DNAm_df = DNAm_df[DNAm_df['duration'] != 0].drop(columns='case_id').reset_index(drop=True)

In [ ]:
final2_CNV_df = pd.concat([final_feats['label'], final_CNV_df], axis=1)
final2_mRNA_df = pd.concat([final_feats['label'], final_mRNA_df], axis=1)
final2_miRNA_df = pd.concat([final_feats['label'], final_miRNA_df], axis=1)
final2_DNAm_df = pd.concat([final_feats['label'], final_DNAm_df], axis=1)


In [ ]:
final2_CNV_df.to_csv("/Users/oo/Desktop/final omics data/validation data/CNV.csv")
final2_mRNA_df.to_csv("/Users/oo/Desktop/final omics data/validation data/mRNA.csv")
final2_miRNA_df.to_csv("/Users/oo/Desktop/final omics data/validation data/miRNA.csv")
final2_DNAm_df.to_csv("/Users/oo/Desktop/final omics data/validation data/DNAm.csv")
